### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [3]:
### Open Source models--Llama3,Gemma2,mistral--Groq
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Load Groq API key from environment variables
groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [4]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001874FB7ED10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001874FB7DA80>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [6]:
# Prepare messages for translation
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello, how are you?")
]

result = model.invoke(messages)

In [7]:
result

AIMessage(content='**Bonjour, comment allez-vous ?** \n\n\nThis is the most common and formal way to say "Hello, how are you?" in French. \n', response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 22, 'total_tokens': 57, 'completion_time': 0.063636364, 'prompt_time': 0.000151889, 'queue_time': 0.01406169, 'total_time': 0.063788253}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-34ae2f64-d4c5-4dbf-8178-a519cca96acd-0', usage_metadata={'input_tokens': 22, 'output_tokens': 35, 'total_tokens': 57})

In [8]:
# Parse the result using a simple string output parser
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
parsed_result = parser.invoke(result)

# Display the parsed result
print("Translation result:", parsed_result)

Translation result: **Bonjour, comment allez-vous ?** 


This is the most common and formal way to say "Hello, how are you?" in French. 



In [12]:
### Using LCEL- chain the components
# Create a chain with the model and parser
chain=model|parser
chain.invoke(messages)
print("Chained Translation result:", chained_result)

Chained Translation result: Bonjour, comment allez-vous ? 



In [13]:
### Using Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

# Define a generic prompt template for translation
generic_template = "Translate the following into {language}:"

# Create a chat prompt template from the generic template
prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template), ("user", "{text}")]
)


In [15]:
result = prompt.invoke({"language":"French", "text":"Hello Nilesh"})

In [16]:
result.to_messages()

[SystemMessage(content='Translate the following into French:'),
 HumanMessage(content='Hello Nilesh')]

In [17]:
### Chaining together components with LCEL

# Create a chain with the prompt, model, and parser
chain = prompt|model|parser
chain.invoke({"language":"French", "text":"Hello"})

'Bonjour \n'